> **Binary** **classification** **model** : AntsBees  

> **Coding** **Stye**: TypeB

> **Section**: Model Class


In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
import torch.utils.data as data
from pathlib import Path


In [ ]:
data_dir = '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data'
root_dir= '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data'

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/AntsBees')

# ***DataSet Class***

In [ ]:
from util.ImageTransform import ImageTransform
from dsets.dsets import make_path_list
from dsets.dsets import MakeBalancedDataset
from torch.utils.data import DataLoader


train_list = make_path_list(phase='train',root_dir=root_dir)
val_list = make_path_list(phase='val',root_dir=root_dir)

file_list={'train':train_list,'val':val_list}

SIZE = 224
# 標準化する際の各RGBの平均値
MEAN = (0.485, 0.456, 0.406) # ImageNetデータセットの平均値を使用
# 標準化する際の各RGBの標準偏差
STD = (0.229, 0.224, 0.225)  # ImageNetデータセットの標準偏差を使用


# 画像のサイズ、平均値、標準偏差の定数値
size, mean, std = SIZE, MEAN, STD

# MakeDatasetで前処理後の訓練データと正解ラベルを取得
train_dataset = MakeBalancedDataset(
    file_list=file_list, # 訓練データのファイルパス
    ratio_int=True,
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='train',records=300)
# MakeDatasetで前処理後の検証データと正解ラベルを取得
val_dataset = MakeBalancedDataset(
    file_list=file_list, # 検証データのファイルパス
    ratio_int=True,
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='val',records=200)


# ミニバッチのサイズを指定
batch_size = 32

# 訓練用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 検証用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# ***Model Class***

In [ ]:
# 使用可能なデバイス(CPUまたはGPU）を取得する
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


###network function

In [ ]:
from models.networks import network_vgg16

In [ ]:
'''
7. 学習済みのVGG16モデルをロード
'''
from torchvision import models
import torch.nn as nn

class MakeModel():
  
  def __init__(self,isTrain=True,continue_train=False,which_epoch=0,use_cuda=True,):
    
    self.save_dir = 'network'
    self.log_dir = 'logmetrics'
    self.isTrain = isTrain
    self.continue_train = continue_train
    self.use_cuda = use_cuda
    self.which_epoch = which_epoch

    if self.isTrain:
      self.model = network_vgg16()
      self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.001,momentum=0.99)

      if not self.continue_train:
        print('first training')

    if not self.isTrain or self.continue_train:
      self.load_network(self.model,self.which_epoch)
      print('continued train')

    if self.use_cuda:
      self.model.cuda()
      print('we use cuda')

  def load_network(self,network,which_epoch):
      save_filename = 'net_%s.pth' % (which_epoch)
      save_path = os.path.join(self.save_dir, save_filename)
      network.load_state_dict(torch.load(save_path))
      print('load network:',save_path)

  def train(self):
    self.model.train()

  def eval(self):
    self.model.eval()



In [ ]:
model=MakeModel(isTrain=True,continue_train=False,which_epoch=0,use_cuda=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

classifier.6.weight
classifier.6.bias
first training
we use cuda


In [ ]:
print(model.model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)

input_t, label_t = batch_tup

input_g=input_t.to(device)
label_g=label_t.to(device)
#model = model.model.to(device)

outputs = model.model(input_g)

print(outputs)

tensor([[ 1.5914e-01,  1.0477e-01],
        [-9.5033e-01,  5.5085e-01],
        [-3.1238e-01, -1.2753e-01],
        [ 6.8136e-01, -1.0557e-01],
        [ 8.0158e-01,  1.0372e+00],
        [-4.6096e-01,  1.1775e-01],
        [ 3.6722e-01,  4.3062e-01],
        [ 3.3585e-01,  5.8120e-01],
        [ 5.9060e-02,  7.7534e-01],
        [ 2.0432e-01,  4.3107e-01],
        [-6.6034e-01, -9.2191e-02],
        [-2.3881e-02,  3.3352e-01],
        [ 4.1836e-01, -5.4855e-01],
        [-2.9658e-01,  1.6987e-01],
        [-3.6296e-04, -1.0090e-01],
        [ 7.0883e-02,  1.0445e-01],
        [ 7.2449e-01, -4.1259e-02],
        [ 1.0820e+00,  8.7261e-01],
        [ 3.7070e-01,  7.4949e-01],
        [ 2.6136e-01,  5.6178e-01],
        [-3.3872e-01,  1.5412e-01],
        [ 6.5810e-01,  1.1404e+00],
        [-1.7782e-01,  9.5781e-01],
        [ 9.6574e-01, -3.2316e-01],
        [ 9.7143e-01,  1.0245e-01],
        [ 3.8940e-01,  4.3495e-01],
        [ 2.7255e-01, -1.9042e-01],
        [-5.3861e-01, -1.754

In [ ]:


pred_label= outputs.argmax(dim=-1)
print(pred_label)
pred_label_list = pred_label.tolist()

print(pred_label_list)

print(outputs)
print(outputs.argmax(dim=-1).tolist())
print(label_t.tolist())
from sklearn.metrics import accuracy_score
print(accuracy_score(label_t.tolist(),pred_label.tolist()))



In [ ]:
softmax = nn.Softmax(dim=1)
prob_g=softmax(outputs)
print(prob_g[:,1])

tensor([0.3567, 0.2787, 0.3057, 0.3406, 0.5154, 0.2812, 0.5654, 0.0557, 0.3600,
        0.3815, 0.3461, 0.2706, 0.4195, 0.4139, 0.5198, 0.1794, 0.7709, 0.6084,
        0.4771, 0.1466, 0.5732, 0.6261, 0.3518, 0.7501, 0.4748, 0.4381, 0.5065,
        0.6065, 0.6254, 0.6377, 0.6341, 0.5592], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [ ]:
loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g).detach()
print(loss_g)

tensor([0.4412, 1.2776, 0.3648, 1.0770, 0.7245, 0.3302, 0.8332, 2.8872, 1.0218,
        0.9636, 1.0610, 1.3070, 0.8687, 0.5342, 0.6543, 1.7179, 0.2602, 0.4969,
        0.6484, 0.1585, 0.5565, 0.4683, 0.4336, 0.2875, 0.7448, 0.8254, 0.6803,
        0.9326, 0.4694, 1.0153, 0.4556, 0.8191], device='cuda:0')


#END